<a href="https://colab.research.google.com/github/neostrange/CveParse/blob/main/CVE_to_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install panda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7256 sha256=303c8bb2c374aae7b481db24ab88ed9cdab0b5cdbff0355b75779356344ae89b
  Stored in directory: /root/.cache/pip/wheels/e9/b8/cc/e2f18ad77320b551f1b7435014844a9cc3e9d6185cb93a3a31
Successfully built panda


In [16]:
!nano /sample_data/csv_dataset_partitive.csv

/bin/bash: nano: command not found


In [22]:
import pandas as pd

In [ ]:
pd.read_csv('sample_data/csv_dataset_partitive.csv')

In [2]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 157 kB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 174 kB 44.8 MB/s 
  Created wheel for neo4j: filename=neo4j-5.2.1-py3-none-any.whl size=249499 sha256=bf8d67fe5572d9597ec83580a67ff24ffad4dc5bd99fe2cd5e354800163df6ad
  Stored in directory: /root/.cache/pip/wheels/3f/cb/09/5c56a88c1650f9d4fe57e59d9a67a5d77fcef88f55d5c233f1
Successfully built neo4j


In [3]:
from neo4j import GraphDatabase

In [4]:

uri = "neo4j+s://14c06ac2.databases.neo4j.io"
pwd = "ySqXDlOHgSJpamhJtvyFNXwgQxSzNhyz1CxdD2MeXR4"

driver = GraphDatabase.driver(uri, auth=("neo4j", pwd))


In [5]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [6]:
uri = "neo4j+s://14c06ac2.databases.neo4j.io"
pwd = "ySqXDlOHgSJpamhJtvyFNXwgQxSzNhyz1CxdD2MeXR4"
user = 'neo4j'
conn = Neo4jConnection(uri, user, pwd)

In [19]:
result = conn.query('MATCH (n) RETURN COUNT(n) AS ct')
print(result)

[<Record ct=0>]


In [21]:
csvFile = 'https://github.com/neostrange/CveParse/blob/main/csv_dataset_partitive.csv'

explore_reviews_csv_query = """
LOAD CSV with headers FROM "https://raw.githubusercontent.com/neostrange/CveParse/main/csv_dataset_partitive.csv" AS row
WITH row WHERE row.ID IS NOT NULL
merge (e:CVE_Entry {ID: row.ID, Status: row.Status
, Phase: row.Phase, Votes: row.Votes})
SET e.Comments = CASE trim(row.Comments) WHEN "" THEN null ELSE row.Comments END
SET e.Description = CASE trim(row.Description) WHEN "" THEN null ELSE row.Description END
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]
